In [1]:
import os
import csv
import math
import wandb
import numpy as np
from pathlib import Path 

from keras.utils import np_utils
from keras.datasets import mnist
from keras.models import Sequential
from wandb.keras import WandbCallback
from keras.layers import Dense, Flatten, Dropout

Using TensorFlow backend.


In [2]:
#Getting the latest file from the folder
record_path = Path("RecordSummary")
training_data_list = list(filter(os.path.isfile,record_path.glob('*.csv')))
training_data_list.sort(key=lambda x: os.path.getmtime(x))
training_data = training_data_list[-1]

x_train_percent = 60
x_test_percent = 100-x_train_percent
x_train = []
x_test = []
y_train = []
y_test = []

with open(training_data) as csv_file:
    data = list(csv.reader(csv_file))
    lines = list(map(list, zip(*data)))

nr_x_train = math.ceil(len(lines[0])*x_train_percent/100)
lines = np.asarray(lines)


#X stands for input, Y stands for output

x_train = lines[1:, 0:nr_x_train]
y_train = lines[0, 0:nr_x_train]

x_test = lines[1:, nr_x_train:] 
y_test = lines[0, nr_x_train:]
x_train = x_train.transpose()
x_test = x_test.transpose()

x_train = [a.reshape(4,96) for a in x_train]
x_test = [a.reshape(4,96) for a in x_test]

is_up_y_train = y_train=='up'
is_up_y_test = y_test=='up'
labels = ["Not UP", "UP"]

In [4]:
channels = np.shape(x_train)[1]
samples = np.shape(x_train)[2]

print(channels)
print(samples)

4
96


In [5]:
print(np.shape(x_train))
print(np.shape(is_up_y_train))
print(np.shape(x_test))
print(np.shape(is_up_y_test))

(132, 4, 96)
(132,)
(87, 4, 96)
(87,)


In [6]:
print(type(x_train))
print(type(is_up_y_train))

<class 'list'>
<class 'numpy.ndarray'>


In [7]:
#converting <class 'list'> to <class 'numpy.ndarray'>
x_train = np.asarray(x_train)
x_test = np.asarray(x_test)

In [20]:
# create model
model=Sequential()
model.add(Flatten(input_shape=(channels,samples)))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='mse', optimizer='adam',
                metrics=['accuracy'])

In [21]:
# logging code
run = wandb.init(project="eeg-up-down")
config = run.config
config.epochs = 10
# Fit the model
model.fit(x_train, is_up_y_train, epochs=config.epochs, validation_data=(x_test, is_up_y_test),
                    callbacks=[WandbCallback(labels=labels)])

Train on 132 samples, validate on 87 samples
Epoch 1/10
132/132 [==============================] - 0s 665us/step - loss: 1163.6202 - accuracy: 0.0076 - val_loss: 2820.7586 - val_accuracy: 0.0000e+00
Epoch 2/10
132/132 [==============================] - 0s 303us/step - loss: 1042.0079 - accuracy: 0.0000e+00 - val_loss: 2365.1410 - val_accuracy: 0.0115
Epoch 3/10
132/132 [==============================] - 0s 370us/step - loss: 485.3487 - accuracy: 0.0227 - val_loss: 1889.7678 - val_accuracy: 0.0575
Epoch 4/10
132/132 [==============================] - 0s 303us/step - loss: 446.9667 - accuracy: 0.0303 - val_loss: 2047.2350 - val_accuracy: 0.0345
Epoch 5/10
132/132 [==============================] - 0s 296us/step - loss: 287.9196 - accuracy: 0.0076 - val_loss: 2238.3613 - val_accuracy: 0.0115
Epoch 6/10
132/132 [==============================] - 0s 296us/step - loss: 235.2560 - accuracy: 0.0227 - val_loss: 2024.1217 - val_accuracy: 0.0460
Epoch 7/10
132/132 [==============================]

In [22]:
print(model.predict(x_test[:100,:,:]))
print(is_up_y_test[:100])

[[-2.3008373e+00]
 [-1.1689644e+01]
 [-6.6963369e-01]
 [-1.1389257e+01]
 [ 1.3056808e+00]
 [-1.0227908e+01]
 [ 1.0793037e+01]
 [-4.9097533e+00]
 [-1.3539125e-02]
 [ 1.5840045e+00]
 [ 1.9954836e+01]
 [-5.6476331e+00]
 [-7.9740411e-01]
 [ 7.8677001e+00]
 [-6.0992613e+00]
 [ 7.6406183e+00]
 [-1.4299195e+00]
 [ 7.1601629e+00]
 [-1.8812613e+01]
 [ 4.1656165e+00]
 [-1.4653268e+00]
 [ 1.9256714e+01]
 [-5.0092139e+00]
 [ 5.4463309e-01]
 [ 1.2877664e+01]
 [-3.7976894e+01]
 [-1.5220685e+01]
 [ 2.1040885e+00]
 [ 2.3051067e+01]
 [-3.0924711e+01]
 [ 5.1582174e+00]
 [-1.6089090e+01]
 [-1.8726084e+01]
 [-8.8039923e+00]
 [-6.8157238e-01]
 [-7.7849236e+00]
 [ 8.1518412e-01]
 [-1.3696944e+01]
 [-4.2664796e-01]
 [-1.0022473e+01]
 [ 7.2021216e-01]
 [-7.9170551e+00]
 [ 7.6011391e+00]
 [ 2.8375227e+00]
 [-3.0631858e-01]
 [-9.0021973e+00]
 [-1.0166352e+01]
 [ 3.2787802e+00]
 [ 1.5260325e+01]
 [ 3.1198256e+00]
 [ 2.3083847e+00]
 [-6.0218434e+00]
 [-9.5355827e-01]
 [-3.6527306e+01]
 [ 7.2727017e+00]
 [ 1.05378